# Heuristics - Flowshop problem

This notebook contains a hands-on the flowshop problem. We will focus on implementing some of the most known heuristics to solve this problem. These heuristics can quickly generate good solutions, even for large problem instances, with a reasonable amount of computational resources. This is particularly useful in production environments where quick turnaround times are important.

### Table of content
- [NEH Heuristic](#NEH)
- [Greedy NEH](#Greedy-NEH)
- [Johnson Heuristic](#Johnson)
- [Ham Heuristic](#Ham)
- [Palmer Heuristic](#Palmer)
- [CDS Heuristic](#CDS)
- [Gupta Heuristic](#Gupta)
- [PRSKE Heuristic](#PRSKE)
- [Artificial Heuristic](#Artificial-Heuristic)
- [Tests](#Tests)

### References
- [Finding an Optimal Sequence in the Flowshop Scheduling Using Johnson’s Algorithm](https://ijiset.com/vol2/v2s1/IJISET_V2_I1_50.pdf)
- [Benchmarks for Basic Scheduling Problems](http://mistic.heig-vd.ch/taillard/articles.dir/Taillard1993EJOR.pdf)
- [Selected heuristic algorithms for solving job shop and flow shop scheduling problems](https://core.ac.uk/display/53190359?utm_source=pdf&utm_medium=banner&utm_campaign=pdf-decoration-v1)

In [1]:
import numpy as np
import matplotlib as plt
import itertools
import time
import pandas as pd
import random

In [2]:
def evaluate_sequence(sequence, processing_times):
    _, num_machines = processing_times.shape
    num_jobs = len(sequence)
    completion_times = np.zeros((num_jobs, num_machines))
    
    # Calculate the completion times for the first machine
    completion_times[0][0] = processing_times[sequence[0]][0]
    for i in range(1, num_jobs):
        completion_times[i][0] = completion_times[i-1][0] + processing_times[sequence[i]][0]
    
    # Calculate the completion times for the remaining machines
    for j in range(1, num_machines):
        completion_times[0][j] = completion_times[0][j-1] + processing_times[sequence[0]][j]
        for i in range(1, num_jobs):
            completion_times[i][j] = max(completion_times[i-1][j], completion_times[i][j-1]) + processing_times[sequence[i]][j]
    
    # Return the total completion time, which is the completion time of the last job in the last machine
    return completion_times[num_jobs-1][num_machines-1]

# NEH

In [3]:
def order_jobs_in_descending_order_of_total_completion_time(processing_times):
    total_completion_time = processing_times.sum(axis=1)
    return np.argsort(total_completion_time, axis=0).tolist()

In [4]:
def insertion(sequence, position, value):
    new_seq = sequence[:]
    new_seq.insert(position, value)
    return new_seq

In [5]:
def neh_algorithm(processing_times):
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    # Define the initial order
    J1, J2 = ordered_sequence[:2]
    sequence = [J1, J2] if evaluate_sequence([J1, J2], processing_times) < evaluate_sequence([J2, J1], processing_times) else [J2, J1]
    del ordered_sequence[:2]
    # Add remaining jobs
    for job in ordered_sequence:
        Cmax = float('inf')
        best_sequence = []
        for i in range(len(sequence)+1):
            new_sequence = insertion(sequence, i, job)
            Cmax_eval = evaluate_sequence(new_sequence, processing_times)
            if Cmax_eval < Cmax:
                Cmax = Cmax_eval
                best_sequence = new_sequence
        sequence = best_sequence
    return sequence, Cmax

# Greedy NEH

In [6]:
def greedy_neh_algorithm(processing_times, num_candidates, num_iterations):
    n = len(processing_times)
    ordered_sequence = order_jobs_in_descending_order_of_total_completion_time(processing_times)
    best_sequence = []
    best_cmax = float('inf')
    for i in range(num_iterations):
        partial_sequence = [ordered_sequence[0]]
        for k in range(1, n):
            candidates = []
            for job in ordered_sequence:
                if job not in partial_sequence:
                    for i in range(k+1):
                        candidate_sequence = insertion(partial_sequence, i, job)
                        candidate_cmax = evaluate_sequence(candidate_sequence, processing_times)
                        candidates.append((candidate_sequence, candidate_cmax))
            candidates.sort(key=lambda x: x[1])
            partial_sequence, cmax = random.choice(candidates[:num_candidates])
        if cmax < best_cmax:
                best_sequence = partial_sequence
                best_cmax = cmax
        ordered_sequence.append(ordered_sequence.pop(0))
    return best_sequence, best_cmax

# Johnson

In [7]:
def johnson_method(processing_times):
    jobs, machines = processing_times.shape
    copy_processing_times = processing_times.copy()
    maximum = processing_times.max() + 1
    m1 = []
    m2 = []
    
    if machines != 2:
        raise Exception("Johson method only works with two machines")
        
    for i in range(jobs):
        minimum = copy_processing_times.min()
        position = np.where(copy_processing_times == minimum)
        
        if position[1][0] == 0:
            m1.append(position[0][0])
        else:
            m2.insert(0, position[0][0])
        
        copy_processing_times[position[0][0]] = maximum
        
    return m1+m2

# Ham

In [8]:
def ham_heuristic(processing_time):
    jobs, machines = processing_time.shape
    sequence = list(range(jobs))
    # Calculating the first summation
    P1 = processing_time[:,:machines//2].sum(axis=1)
    # Calculating the second summation
    P2 = processing_time[:,machines//2:].sum(axis=1)
    # Calculating the first solution, ordered by P2 - P1
    P2_P1 = P2 - P1
    solution_1 = [job for _ , job in sorted(zip(P2_P1, sequence), reverse=True)]
    # Calculating the second solution
    positives = np.argwhere(P2_P1 >= 0).flatten()
    negatives = np.argwhere(P2_P1 < 0).flatten()
    positive_indices = [job for _ , job in sorted(zip(P1[positives], positives))]
    negative_indices = [job for _ , job in sorted(zip(P2[negatives], negatives), reverse=True)]
    positive_indices.extend(negative_indices)
    # Calculating Cmax for both solutions
    Cmax1 = evaluate_sequence(solution_1, processing_time)
    Cmax2 = evaluate_sequence(positive_indices, processing_time)
    # Returning the best solution among them
    if Cmax1 < Cmax2:
        return solution_1, Cmax1
    else:
        return positive_indices, Cmax2

# Palmer

In [9]:
def palmer_heuristic(processing_times):
    jobs, machines = processing_times.shape
    slope_indices = []
    for i in range(jobs):
        processing_time_sum = np.sum(processing_times[i])
        fi = 0
        for j in range(machines):
            fi += (machines - 2*j + 1) * processing_times[i][j] / processing_time_sum
        slope_indices.append(fi)
    order = sorted(range(jobs), key=lambda k: slope_indices[k])
    return order

# CDS

In [10]:
def CDS_heuristic(processing_times):
    jobs, machines = processing_times.shape
    m = machines-1
    johnson_proc_times = np.zeros((jobs,2))
    best_cost = np.inf
    best_seq = []
    for k in range(m):
        johnson_proc_times[:,0] += processing_times[:,k]
        johnson_proc_times[:,1] += processing_times[:,-k-1]
        seq = johnson_method(johnson_proc_times)
        cost = evaluate_sequence(seq,processing_times)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
    return best_seq, best_cost

# Gupta

In [11]:
def sign(x):
    if x > 0:
        return 1
    elif x < 0:
        return -1
    else:
        return 0

In [12]:
def min_gupta(job, processing_times):
    m = np.inf
    _, machines = processing_times.shape
    for i in range(machines-1):
        k = processing_times[job][i] + processing_times[job][i+1]
        if (k < m):
            m = k
    return m

In [13]:
def gupta_heuristic(processing_times):
    jobs, machines = processing_times.shape
    f = []
    total_times = []
    for i in range(jobs):
        fi = sign(processing_times[i][0] - processing_times[i][machines-1]) / min_gupta(i,processing_times)
        f.append(fi)
        total_time = sum(processing_times[i])
        total_times.append(total_time)
    order = sorted(range(jobs), key=lambda k: (f[k], total_times[k]))
    return order

# PRSKE

In [14]:
def skewness(processing_times):
    jobs, machines = processing_times.shape
    skewnesses = []
    # Calculate the skewness for each job 
    for i in range(jobs):
        avg = np.mean(processing_times[i,:])
        numerator = 0
        denominator = 0
        for j in range(machines):
            m = (processing_times[i,j] - avg)
            numerator += m**3
            denominator += m**2
        # Actually calculating the skewness    
        numerator = numerator*(1/machines)
        denominator = (np.sqrt(denominator*(1/machines)))**3
        skewnesses.append(numerator/denominator)
    return np.array(skewnesses)

In [15]:
def PRSKE_heuristic(processing_times):
    avg = np.mean(processing_times, axis=1)
    std = np.std(processing_times, axis=1, ddof=1)
    skw = skewness(processing_times)
    order = skw + std + avg
    sequence = [job for _ , job in sorted(zip(order, list(range(processing_times.shape[0]))),reverse=True)]
    return sequence, evaluate_sequence(sequence, processing_times)

# Artificial Heuristic

In [16]:
def artificial_heuristic(processing_times):
    jobs, machines = processing_times.shape
    r = 1
    best_cost = np.inf
    best_seq = []
    while r != machines :
        wi = np.zeros((jobs, machines - r))
        for i in range(jobs):
            for j in range(0, machines - r):
                wi[i, j] = (machines - r) - (j)
       
        am = np.zeros((jobs, 2))    
        am[:, 0] = np.sum(wi[:, :machines - r] * processing_times[:, :machines - r], axis=1)
        for i in range(jobs):
            for j in range(0, machines - r):
                am[i, 1] += wi[i, j ] * processing_times[i, machines - j - 1]
        seq = johnson_method(am)
        cost = evaluate_sequence(seq, processing_times)
        if cost < best_cost:
            best_cost = cost
            best_seq = seq
        r += 1
       
    return best_seq, best_cost

In [17]:
def all_permutations(iterable):
    permutations = list(itertools.permutations(iterable))
    permutations_as_lists = [list(p) for p in permutations]
    return permutations_as_lists

In [18]:
def brute_force(processing_times, permutations):
    M = float('inf')
    sol = []
    for permutation in permutations:
        m = evaluate_sequence(permutation, processing_times)
        if m < M:
            M = m
            sol = permutation
    return sol, M

In [19]:
rnd_data = np.random.randint(size=(8,5), low=5, high=120)
print(rnd_data, "\n")

[[ 93  58  12 115  58]
 [ 87 110  32  98  60]
 [ 66  84  93  44  49]
 [ 72 107  77  62  21]
 [119 107  94  67  83]
 [ 55  82  55  16  53]
 [ 60  56  24  97  40]
 [ 73   6 100  41  12]] 



In [20]:
init_jobs = 8
init_job_list = list(range(init_jobs))

start_time = time.time()
sequence_list = all_permutations(init_job_list)
sol, M = brute_force(rnd_data, sequence_list)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Brute Force {sol} with a makespan of {M}')
print("Elapsed time:", elapsed_time, "seconds")

Best sequence found by Brute Force [1, 4, 0, 2, 3, 5, 6, 7] with a makespan of 877.0
Elapsed time: 5.902706623077393 seconds


In [21]:
start_time = time.time()
sol = gupta_heuristic(rnd_data)
M = evaluate_sequence(sol, rnd_data)
elapsed_time = time.time() - start_time

print(f'Best sequence found by Brute Force {sol} with a makespan of {M}')
print("Elapsed time:", elapsed_time, "seconds")

Best sequence found by Brute Force [4, 1, 2, 3, 6, 0, 5, 7] with a makespan of 1001.0
Elapsed time: 0.0004127025604248047 seconds


# Tests

In this notebook, we will test all the heuristics implemented above on the first and the seventh instance of each benchmark listed below:
- Taillard, 20 jobs 5 machines.
- Taillard, 50 jobs 10 machines.
- Taillard, 100 jobs 10 machines.
- Taillard, 200 jobs 10 machines.

For each test, we will save the execution time and the deviation.
The deviation being equal to $\frac{Cmax-UP}{UP}$.

In [22]:
def deviation(Cmax, UP):
    return ((Cmax - UP)/UP)*100

## Reading Taillard Instances

### Taillard, 20 jobs 5 machines

In [23]:
# Open the file that contains the instances
file = open("Benchmarks/tai20_5.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_20_5 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_20_5[n].append(line)
    else:
        instances_20_5.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 20 jobs 5 machines contains {len(instances_20_5)} benchmark.')   

Taillard, 20 jobs 5 machines contains 10 benchmark.


### Taillard, 50 jobs 10 machines

In [24]:
# Open the file that contains the instances
file = open("Benchmarks/tai50_10.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_50_10 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_50_10[n].append(line)
    else:
        instances_50_10.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 50 jobs 10 machines contains {len(instances_50_10)} benchmark.')

Taillard, 50 jobs 10 machines contains 10 benchmark.


### Taillard, 100 jobs 10 machines

In [25]:
# Open the file that contains the instances
file = open("Benchmarks/tai100_10.txt", "r")

# Read the file line by line to retrieve the instances
n = 0
instances_100_10 = [[]]
line = file.readline()

while line:
    if line != '\n':
        line = line.strip(' ')
        line = line[:-1]
        line = line.split()
        line = [int(num) for num in line]
        instances_100_10[n].append(line)
    else:
        instances_100_10.append([])
        n += 1
    line = file.readline()
    
print(f'Taillard, 100 jobs 10 machines contains {len(instances_100_10)} benchmark.')

Taillard, 100 jobs 10 machines contains 10 benchmark.


## Tests on the 7th instance

In [26]:
# Prepare a dataframe that will gather all our results
df_1 = pd.DataFrame({'Heuristic': pd.Series(dtype='str'),
                   '20-5 (Deviation)': pd.Series(dtype='str'),
                   '20-5 (Time)': pd.Series(dtype='str'),
                   '50-10 (Deviation)': pd.Series(dtype='str'),
                   '50-10 (Time)': pd.Series(dtype='str'),
                   '100-10 (Deviation)': pd.Series(dtype='str'),
                   '100-10 (Time)': pd.Series(dtype='str'),
                  }) 

In [27]:
NEH = ['NEH']
GNEH = ['Greedy NEH']
Ham = ['Ham']
Gupta = ['Gupta']
Palmer = ['Palmer']
CDS = ['CDS']
PRSKE = ['PRSKE']
ARTIFICIAL = ['Artificial Heuristic']

### 20 jobs 5 machines

In [28]:
instance_0 = np.array(instances_20_5[6])
instance_0 = instance_0.T

In [29]:
instance_0

array([[15, 28, 77,  1, 45],
       [64,  4, 36, 59, 73],
       [64, 43, 57, 95, 59],
       [48, 93, 15, 49, 63],
       [ 9,  1, 81, 90, 54],
       [91, 81, 82, 78, 98],
       [27, 77, 98,  3, 39],
       [34, 69, 97, 69, 75],
       [42, 52, 12, 99, 33],
       [ 3, 28, 35, 41,  8],
       [11, 28, 84, 73, 86],
       [54, 77, 70, 28, 41],
       [27, 42, 27, 99, 41],
       [30, 53, 37, 13, 22],
       [ 9, 46, 59, 59, 43],
       [15, 49, 42, 47, 34],
       [88, 15, 57,  8, 80],
       [55, 43, 16, 92, 16],
       [50, 65, 11, 87, 37],
       [57, 41, 34, 62, 94]])

In [30]:
print(evaluate_sequence([2, 19, 15, 1, 4, 3, 10, 5, 12, 16, 7, 18, 14, 6, 13, 8, 0, 17, 11, 9],instance_0))

1324.0


In [31]:
instance_0 = np.array(instances_20_5[6])
instance_0 = instance_0.T
UB = 1239

In [32]:
# NEH Algorithm
start_time = time.time()
sol, Cmax = neh_algorithm(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

NEH.append("%.3f" % (deviation(Cmax, UB)))
NEH.append(" %.3fs" % (elapsed_time))

Solution: [4, 2, 19, 10, 7, 5, 3, 8, 1, 12, 6, 18, 16, 9, 14, 15, 0, 17, 13, 11].

Cmax: 1284.0

Elapsed time:  0.0337371826171875 seconds


In [33]:
# Greedy NEH Algorithm
start_time = time.time()
sol, Cmax = greedy_neh_algorithm(instance_0, 4, 6)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

GNEH.append("%.3f" % (deviation(Cmax, UB)))
GNEH.append(" %.3fs" % (elapsed_time))

Solution: [4, 15, 1, 10, 14, 5, 11, 19, 2, 7, 8, 6, 17, 13, 0, 16, 3, 12, 18, 9].

Cmax: 1257.0

Elapsed time:  1.8922481536865234 seconds


In [34]:
# Ham Heuristic
start_time = time.time()
sol, Cmax = ham_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")


Ham.append("%.3f" % (deviation(Cmax, UB)))
Ham.append(" %.3fs" % (elapsed_time))

Solution: [4, 10, 7, 14, 2, 1, 12, 19, 5, 0, 15, 9, 8, 16, 6, 17, 18, 11, 13, 3].

Cmax: 1334.0

Elapsed time:  0.005689144134521484 seconds


In [35]:
# Gupta Heuristic
start_time = time.time()
sol = gupta_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Gupta.append("%.3f" % (deviation(Cmax, UB)))
Gupta.append(" %.3fs" % (elapsed_time))

Solution: [4, 9, 10, 1, 6, 0, 14, 15, 3, 12, 19, 7, 5, 2, 18, 11, 16, 8, 17, 13].

Cmax: 1390.0

Elapsed time:  0.0020165443420410156 seconds


In [36]:
# Palmer Heuristic
start_time = time.time()
sol = palmer_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Palmer.append("%.3f" % (deviation(Cmax, UB)))
Palmer.append(" %.3fs" % (elapsed_time))

Solution: [4, 10, 14, 12, 19, 1, 7, 9, 0, 15, 2, 8, 5, 18, 3, 16, 17, 6, 11, 13].

Cmax: 1360.0

Elapsed time:  0.004403352737426758 seconds


In [37]:
# CDS Heuristic
start_time = time.time()
sol, Cmax = CDS_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

CDS.append("%.3f" % (deviation(Cmax, UB)))
CDS.append(" %.3fs" % (elapsed_time))

Solution: [9, 4, 14, 10, 0, 15, 6, 12, 7, 3, 19, 1, 5, 16, 2, 11, 18, 8, 13, 17].

Cmax: 1393.0

Elapsed time:  0.00465083122253418 seconds


In [38]:
# PKSE Heuristic
start_time = time.time()
sol, Cmax = PRSKE_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

PRSKE.append("%.3f" % (deviation(Cmax, UB)))
PRSKE.append(" %.3fs" % (elapsed_time))

Solution: [5, 7, 10, 4, 6, 16, 2, 3, 19, 8, 18, 12, 17, 1, 11, 0, 14, 15, 13, 9].

Cmax: 1636.0

Elapsed time:  0.0009961128234863281 seconds


In [39]:
# Artificial_heuristic Heuristic
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

ARTIFICIAL.append("%.3f" % (deviation(Cmax, UB)))
ARTIFICIAL.append(" %.3fs" % (elapsed_time))

Solution: [9, 4, 14, 10, 0, 15, 6, 12, 7, 3, 19, 1, 5, 16, 2, 11, 18, 8, 13, 17].

Cmax: 1393.0

Elapsed time:  0.004266023635864258 seconds


### 50 jobs 10 machines

In [40]:
instance_0 = np.array(instances_50_10[6])
instance_0 = instance_0.T
UB = 3107

In [41]:
# NEH Algorithm
start_time = time.time()
sol, Cmax = neh_algorithm(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

NEH.append("%.3f" % (deviation(Cmax, UB)))
NEH.append(" %.3fs" % (elapsed_time))

Solution: [48, 0, 36, 18, 7, 4, 9, 44, 2, 1, 8, 19, 34, 27, 41, 13, 6, 3, 15, 11, 29, 43, 22, 20, 25, 47, 42, 23, 26, 32, 16, 12, 5, 31, 28, 38, 46, 37, 21, 33, 40, 39, 45, 49, 10, 14, 24, 17, 30, 35].

Cmax: 3271.0

Elapsed time:  1.734560251235962 seconds


In [42]:
# Greedy NEH Algorithm
start_time = time.time()
sol, Cmax = greedy_neh_algorithm(instance_0, 4, 1)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

GNEH.append("%.3f" % (deviation(Cmax, UB)))
GNEH.append(" %.3fs" % (elapsed_time))

Solution: [16, 19, 1, 36, 9, 8, 46, 18, 42, 27, 44, 0, 6, 23, 4, 10, 22, 20, 32, 15, 11, 41, 34, 29, 43, 14, 49, 2, 21, 26, 5, 47, 13, 39, 17, 25, 7, 48, 31, 3, 38, 33, 28, 24, 12, 30, 45, 37, 35, 40].

Cmax: 3273.0

Elapsed time:  21.662691831588745 seconds


In [43]:
# Ham Heuristic
start_time = time.time()
sol, Cmax = ham_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Ham.append("%.3f" % (deviation(Cmax, UB)))
Ham.append(" %.3fs" % (elapsed_time))

Solution: [5, 40, 47, 31, 32, 36, 48, 26, 16, 33, 19, 1, 9, 0, 27, 6, 42, 8, 7, 10, 15, 18, 4, 13, 46, 49, 34, 23, 43, 2, 14, 29, 41, 28, 44, 24, 11, 30, 45, 38, 3, 20, 39, 21, 22, 35, 12, 25, 17, 37].

Cmax: 3520.0

Elapsed time:  0.008232355117797852 seconds


In [44]:
# Gupta Heuristic
start_time = time.time()
sol = gupta_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Gupta.append("%.3f" % (deviation(Cmax, UB)))
Gupta.append(" %.3fs" % (elapsed_time))

Solution: [32, 31, 25, 40, 47, 20, 39, 15, 41, 12, 1, 36, 0, 9, 8, 27, 2, 7, 18, 26, 13, 42, 34, 49, 10, 29, 23, 3, 4, 6, 38, 19, 28, 24, 22, 45, 44, 16, 14, 46, 43, 48, 11, 30, 35, 37, 17, 5, 21, 33].

Cmax: 3696.0

Elapsed time:  0.006766080856323242 seconds


In [45]:
# Palmer Heuristic
start_time = time.time()
sol = palmer_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Palmer.append("%.3f" % (deviation(Cmax, UB)))
Palmer.append(" %.3fs" % (elapsed_time))

Solution: [36, 31, 48, 8, 0, 47, 9, 1, 32, 19, 27, 5, 18, 42, 7, 15, 6, 41, 26, 43, 16, 45, 4, 2, 20, 33, 34, 10, 40, 11, 25, 13, 39, 23, 44, 49, 28, 17, 14, 46, 12, 22, 29, 24, 21, 30, 3, 38, 35, 37].

Cmax: 3543.0

Elapsed time:  0.01179361343383789 seconds


In [46]:
# CDS Heuristic
start_time = time.time()
sol, Cmax = CDS_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

CDS.append("%.3f" % (deviation(Cmax, UB)))
CDS.append(" %.3fs" % (elapsed_time))

Solution: [5, 40, 47, 31, 32, 36, 48, 26, 16, 33, 19, 1, 9, 0, 27, 6, 42, 8, 7, 10, 15, 18, 4, 13, 46, 49, 34, 23, 43, 2, 14, 29, 41, 28, 44, 24, 11, 30, 45, 38, 3, 20, 39, 21, 22, 35, 12, 25, 17, 37].

Cmax: 3520.0

Elapsed time:  0.048005104064941406 seconds


In [47]:
# PKSE Heuristic
start_time = time.time()
sol, Cmax = PRSKE_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

PRSKE.append("%.3f" % (deviation(Cmax, UB)))
PRSKE.append(" %.3fs" % (elapsed_time))

Solution: [4, 31, 38, 13, 46, 11, 49, 44, 23, 3, 27, 18, 24, 15, 36, 8, 34, 42, 41, 33, 10, 14, 21, 7, 28, 0, 19, 43, 6, 29, 48, 37, 22, 2, 35, 30, 9, 12, 17, 20, 47, 1, 45, 32, 16, 25, 5, 39, 40, 26].

Cmax: 3804.0

Elapsed time:  0.010255575180053711 seconds


In [48]:
# Artificial_heuristic Heuristic
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

ARTIFICIAL.append("%.3f" % (deviation(Cmax, UB)))
ARTIFICIAL.append(" %.3fs" % (elapsed_time))

Solution: [5, 32, 47, 48, 26, 16, 36, 1, 31, 19, 33, 8, 0, 9, 45, 27, 41, 43, 6, 15, 7, 42, 18, 4, 13, 49, 46, 34, 23, 2, 28, 10, 38, 44, 11, 24, 3, 29, 14, 20, 21, 12, 30, 22, 17, 39, 37, 35, 40, 25].

Cmax: 3467.0

Elapsed time:  0.046674489974975586 seconds


### 100 jobs 10 machines

In [49]:
instance_0 = np.array(instances_100_10[6])
instance_0 = instance_0.T
UB = 5599

In [50]:
# NEH Algorithm
start_time = time.time()
sol, Cmax = neh_algorithm(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

NEH.append("%.3f" % (deviation(Cmax, UB)))
NEH.append(" %.3fs" % (elapsed_time))

Solution: [98, 94, 92, 77, 20, 50, 30, 99, 91, 19, 17, 49, 45, 71, 57, 69, 25, 36, 3, 35, 89, 16, 79, 65, 62, 63, 72, 95, 39, 56, 41, 82, 14, 42, 88, 27, 7, 67, 43, 76, 44, 31, 15, 32, 22, 1, 55, 47, 84, 90, 74, 12, 70, 24, 68, 61, 86, 97, 9, 21, 4, 38, 29, 18, 73, 40, 93, 81, 51, 34, 59, 0, 54, 28, 10, 52, 85, 58, 66, 23, 13, 26, 96, 33, 78, 87, 2, 46, 75, 5, 64, 8, 6, 48, 60, 83, 37, 11, 80, 53].

Cmax: 5719.0

Elapsed time:  12.799433469772339 seconds


In [51]:
# Greedy NEH Algorithm
start_time = time.time()
sol, Cmax = greedy_neh_algorithm(instance_0, 2, 1)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

GNEH.append("%.3f" % (deviation(Cmax, UB)))
GNEH.append(" %.3fs" % (elapsed_time))

Solution: [55, 63, 0, 32, 91, 12, 98, 36, 49, 92, 94, 97, 9, 28, 41, 19, 86, 50, 54, 39, 59, 71, 20, 43, 67, 17, 96, 81, 35, 3, 65, 15, 31, 25, 47, 10, 14, 1, 62, 61, 48, 88, 79, 23, 69, 73, 99, 58, 18, 30, 95, 52, 42, 40, 57, 4, 56, 72, 45, 70, 89, 82, 77, 93, 44, 34, 76, 66, 13, 51, 7, 85, 90, 22, 33, 24, 68, 21, 29, 64, 6, 38, 26, 78, 27, 87, 2, 8, 60, 5, 84, 80, 74, 83, 16, 37, 53, 46, 75, 11].

Cmax: 5710.0

Elapsed time:  296.2470271587372 seconds


In [52]:
# Ham Heuristic
start_time = time.time()
sol, Cmax = ham_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Ham.append("%.3f" % (deviation(Cmax, UB)))
Ham.append(" %.3fs" % (elapsed_time))

Solution: [55, 98, 66, 12, 0, 34, 54, 26, 63, 38, 32, 20, 67, 85, 23, 3, 7, 24, 4, 78, 36, 17, 82, 14, 49, 69, 95, 41, 42, 91, 39, 57, 15, 99, 65, 29, 45, 94, 19, 77, 50, 35, 81, 59, 92, 74, 25, 9, 97, 56, 60, 89, 22, 30, 79, 62, 27, 44, 70, 16, 71, 10, 48, 88, 40, 1, 96, 43, 90, 18, 47, 86, 93, 33, 73, 61, 31, 2, 8, 84, 72, 76, 68, 53, 51, 28, 58, 83, 21, 5, 80, 64, 37, 6, 13, 87, 52, 46, 75, 11].

Cmax: 6315.0

Elapsed time:  0.0 seconds


In [53]:
# Gupta Heuristic
start_time = time.time()
sol = gupta_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Gupta.append("%.3f" % (deviation(Cmax, UB)))
Gupta.append(" %.3fs" % (elapsed_time))

Solution: [98, 34, 23, 75, 86, 73, 21, 55, 58, 40, 27, 0, 16, 76, 63, 12, 32, 10, 91, 3, 4, 2, 31, 95, 24, 82, 47, 29, 78, 7, 69, 70, 36, 94, 20, 71, 96, 49, 45, 65, 57, 88, 93, 35, 25, 99, 15, 60, 81, 33, 50, 59, 97, 19, 77, 74, 22, 79, 92, 39, 41, 62, 1, 37, 9, 68, 83, 67, 48, 30, 51, 84, 8, 17, 56, 26, 90, 89, 5, 54, 64, 13, 66, 42, 18, 14, 43, 80, 52, 61, 46, 6, 87, 53, 72, 38, 28, 11, 85, 44].

Cmax: 6524.0

Elapsed time:  0.003027200698852539 seconds


In [54]:
# Palmer Heuristic
start_time = time.time()
sol = palmer_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Palmer.append("%.3f" % (deviation(Cmax, UB)))
Palmer.append(" %.3fs" % (elapsed_time))

Solution: [98, 20, 63, 49, 32, 82, 55, 3, 34, 7, 69, 36, 67, 94, 86, 54, 91, 95, 0, 10, 17, 99, 45, 65, 24, 12, 29, 96, 61, 39, 57, 19, 4, 44, 23, 13, 27, 77, 60, 15, 16, 41, 38, 25, 22, 52, 92, 78, 81, 88, 79, 40, 31, 33, 26, 73, 35, 70, 42, 66, 30, 14, 50, 2, 58, 47, 85, 21, 51, 97, 68, 89, 46, 18, 75, 93, 56, 8, 62, 71, 59, 48, 90, 28, 76, 74, 64, 1, 43, 9, 80, 5, 6, 37, 87, 53, 84, 72, 83, 11].

Cmax: 6280.0

Elapsed time:  0.003830432891845703 seconds


In [55]:
# CDS Heuristic
start_time = time.time()
sol, Cmax = CDS_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

CDS.append("%.3f" % (deviation(Cmax, UB)))
CDS.append(" %.3fs" % (elapsed_time))

Solution: [0, 34, 52, 63, 12, 13, 55, 98, 23, 38, 67, 7, 4, 20, 82, 54, 32, 86, 91, 96, 39, 10, 49, 24, 44, 3, 36, 14, 41, 17, 69, 95, 30, 79, 22, 57, 45, 60, 25, 94, 65, 99, 29, 19, 77, 35, 50, 81, 59, 92, 89, 56, 97, 42, 62, 15, 40, 71, 74, 88, 9, 1, 18, 47, 93, 70, 43, 27, 61, 84, 16, 90, 72, 48, 28, 33, 78, 2, 68, 83, 8, 26, 87, 31, 37, 85, 80, 64, 73, 46, 76, 51, 53, 66, 58, 5, 6, 21, 75, 11].

Cmax: 6201.0

Elapsed time:  0.024932146072387695 seconds


In [56]:
# PKSE Heuristic
start_time = time.time()
sol, Cmax = PRSKE_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

PRSKE.append("%.3f" % (deviation(Cmax, UB)))
PRSKE.append(" %.3fs" % (elapsed_time))

Solution: [89, 35, 29, 81, 94, 77, 9, 16, 72, 92, 74, 71, 56, 40, 84, 93, 91, 99, 62, 50, 59, 1, 83, 43, 76, 42, 97, 44, 19, 95, 39, 30, 32, 15, 22, 49, 36, 20, 65, 45, 28, 14, 67, 25, 18, 17, 57, 79, 73, 47, 90, 86, 87, 61, 10, 60, 69, 27, 3, 63, 7, 48, 54, 80, 70, 8, 41, 68, 96, 5, 58, 88, 2, 82, 23, 53, 33, 6, 37, 46, 64, 31, 0, 4, 21, 24, 98, 38, 78, 11, 26, 34, 85, 12, 66, 55, 13, 51, 75, 52].

Cmax: 6696.0

Elapsed time:  0.0 seconds


In [57]:
# Artificial_heuristic Heuristic
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

ARTIFICIAL.append("%.3f" % (deviation(Cmax, UB)))
ARTIFICIAL.append(" %.3fs" % (elapsed_time))

Solution: [55, 98, 34, 63, 12, 13, 0, 82, 54, 61, 4, 67, 20, 38, 7, 32, 23, 24, 3, 86, 49, 36, 69, 96, 39, 10, 17, 91, 41, 44, 65, 95, 27, 94, 99, 15, 45, 57, 60, 19, 29, 77, 35, 81, 50, 59, 89, 97, 25, 92, 16, 22, 79, 30, 56, 71, 74, 88, 14, 62, 18, 1, 9, 33, 93, 42, 40, 70, 31, 73, 68, 76, 8, 90, 48, 43, 28, 47, 2, 84, 78, 80, 83, 72, 58, 85, 26, 21, 51, 37, 64, 6, 46, 53, 87, 5, 66, 52, 75, 11].

Cmax: 6229.0

Elapsed time:  0.03738665580749512 seconds


### Results

In [58]:
df_1.loc[0]=NEH
df_1.loc[1]=GNEH
df_1.loc[2]=Ham
df_1.loc[3]=Gupta
df_1.loc[4]=Palmer
df_1.loc[5]=CDS
df_1.loc[6]=PRSKE
df_1.loc[7]=ARTIFICIAL

In [59]:
# Displaying the results that we have got for the seventh instance of each benchmark
df_1

Heuristic 20-5 (Deviation) 20-5 (Time) 50-10 (Deviation)  \
0                   NEH            3.632      0.034s             5.278   
1            Greedy NEH            1.453      1.892s             5.343   
2                   Ham            7.667      0.006s            13.293   
3                 Gupta           12.187      0.002s            18.957   
4                Palmer            9.766      0.004s            14.033   
5                   CDS           12.429      0.005s            13.293   
6                 PRSKE           32.042      0.001s            22.433   
7  Artificial Heuristic           12.429      0.004s            11.587   

  50-10 (Time) 100-10 (Deviation) 100-10 (Time)  
0       1.735s              2.143       12.799s  
1      21.663s              1.982      296.247s  
2       0.008s             12.788        0.000s  
3       0.007s             16.521        0.003s  
4       0.012s             12.163        0.004s  
5       0.048s             10.752        0.025s  
6       0.010s             19.593        0.000s  
7       0.047s             11.252        0.037s

## Tests on the 1st instance

In [60]:
# Prepare a dataframe that will gather all our results
df_0 = pd.DataFrame({'Heuristic': pd.Series(dtype='str'),
                   '20-5 (Deviation)': pd.Series(dtype='str'),
                   '20-5 (Time)': pd.Series(dtype='str'),
                   '50-10 (Deviation)': pd.Series(dtype='str'),
                   '50-10 (Time)': pd.Series(dtype='str'),
                   '100-10 (Deviation)': pd.Series(dtype='str'),
                   '100-10 (Time)': pd.Series(dtype='str'),
                  }) 

In [61]:
NEH = ['NEH']
GNEH = ['Greedy NEH']
Ham = ['Ham']
Gupta = ['Gupta']
Palmer = ['Palmer']
CDS = ['CDS']
PRSKE = ['PRSKE']
ARTIFICIAL = ['Artificial Heuristic']

### 20 jobs 5 machines

In [62]:
instance_0 = np.array(instances_20_5[0])
instance_0 = instance_0.T
UB = 1278

In [63]:
# NEH Algorithm
start_time = time.time()
sol, Cmax = neh_algorithm(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

NEH.append("%.3f" % (deviation(Cmax, UB)))
NEH.append(" %.3fs" % (elapsed_time))

Solution: [8, 6, 15, 10, 7, 1, 16, 2, 14, 13, 17, 3, 9, 11, 0, 18, 5, 4, 12, 19].

Cmax: 1334.0

Elapsed time:  0.02990436553955078 seconds


In [64]:
# Greedy NEH Algorithm
start_time = time.time()
sol, Cmax = greedy_neh_algorithm(instance_0, 5, 2)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

GNEH.append("%.3f" % (deviation(Cmax, UB)))
GNEH.append(" %.3fs" % (elapsed_time))

Solution: [16, 8, 14, 3, 0, 10, 9, 7, 2, 17, 18, 6, 15, 5, 13, 4, 19, 11, 1, 12].

Cmax: 1321.0

Elapsed time:  0.33155059814453125 seconds


In [65]:
# Ham Heuristic
start_time = time.time()
sol, Cmax = ham_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")


Ham.append("%.3f" % (deviation(Cmax, UB)))
Ham.append(" %.3fs" % (elapsed_time))

Solution: [2, 8, 16, 14, 18, 10, 1, 12, 15, 7, 13, 5, 0, 4, 9, 17, 3, 6, 19, 11].

Cmax: 1417.0

Elapsed time:  0.0 seconds


In [66]:
# Gupta Heuristic
start_time = time.time()
sol = gupta_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Gupta.append("%.3f" % (deviation(Cmax, UB)))
Gupta.append(" %.3fs" % (elapsed_time))

Solution: [10, 2, 8, 16, 14, 15, 7, 13, 0, 3, 18, 6, 4, 5, 9, 17, 1, 19, 12, 11].

Cmax: 1396.0

Elapsed time:  0.0 seconds


In [67]:
# Palmer Heuristic
start_time = time.time()
sol = palmer_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Palmer.append("%.3f" % (deviation(Cmax, UB)))
Palmer.append(" %.3fs" % (elapsed_time))

Solution: [8, 16, 10, 14, 2, 15, 18, 5, 13, 7, 1, 3, 0, 4, 12, 6, 11, 9, 17, 19].

Cmax: 1384.0

Elapsed time:  0.0 seconds


In [68]:
# CDS Heuristic
start_time = time.time()
sol, Cmax = CDS_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

CDS.append("%.3f" % (deviation(Cmax, UB)))
CDS.append(" %.3fs" % (elapsed_time))

Solution: [14, 2, 8, 13, 16, 7, 6, 0, 18, 3, 10, 15, 11, 1, 4, 5, 19, 17, 9, 12].

Cmax: 1390.0

Elapsed time:  0.003989696502685547 seconds


In [69]:
# PKSE Heuristic
start_time = time.time()
sol, Cmax = PRSKE_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

PRSKE.append("%.3f" % (deviation(Cmax, UB)))
PRSKE.append(" %.3fs" % (elapsed_time))

Solution: [3, 17, 10, 1, 9, 11, 6, 4, 19, 15, 18, 5, 0, 12, 8, 13, 14, 7, 16, 2].

Cmax: 1593.0

Elapsed time:  0.0 seconds


In [70]:
# Artificial_heuristic Heuristic
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

ARTIFICIAL.append("%.3f" % (deviation(Cmax, UB)))
ARTIFICIAL.append(" %.3fs" % (elapsed_time))

Solution: [2, 8, 14, 16, 13, 7, 18, 5, 10, 15, 1, 3, 4, 0, 11, 6, 17, 9, 19, 12].

Cmax: 1367.0

Elapsed time:  0.0 seconds


### 50 jobs 10 machines

In [71]:
instance_0 = np.array(instances_50_10[0])
instance_0 = instance_0.T
UB = 3025

In [72]:
# NEH Algorithm
start_time = time.time()
sol, Cmax = neh_algorithm(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

NEH.append("%.3f" % (deviation(Cmax, UB)))
NEH.append(" %.3fs" % (elapsed_time))

Solution: [19, 33, 30, 14, 35, 13, 41, 34, 9, 43, 32, 42, 24, 11, 39, 46, 17, 47, 12, 40, 37, 8, 4, 45, 28, 1, 2, 10, 44, 21, 5, 3, 48, 16, 31, 29, 25, 49, 23, 0, 15, 36, 7, 27, 6, 22, 20, 38, 26, 18].

Cmax: 3229.0

Elapsed time:  0.7297382354736328 seconds


In [73]:
# Greedy NEH Algorithm
start_time = time.time()
sol, Cmax = greedy_neh_algorithm(instance_0, 4, 1)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

GNEH.append("%.3f" % (deviation(Cmax, UB)))
GNEH.append(" %.3fs" % (elapsed_time))

Solution: [43, 32, 17, 24, 14, 39, 33, 2, 37, 1, 48, 42, 35, 16, 45, 10, 40, 12, 4, 9, 0, 28, 19, 47, 27, 13, 29, 11, 36, 7, 3, 31, 38, 49, 41, 34, 30, 22, 6, 25, 15, 5, 44, 46, 8, 20, 21, 18, 23, 26].

Cmax: 3249.0

Elapsed time:  10.7084481716156 seconds


In [74]:
# Ham Heuristic
start_time = time.time()
sol, Cmax = ham_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Ham.append("%.3f" % (deviation(Cmax, UB)))
Ham.append(" %.3fs" % (elapsed_time))

Solution: [32, 41, 43, 23, 1, 19, 17, 24, 36, 30, 0, 29, 25, 13, 48, 42, 35, 9, 37, 5, 28, 46, 45, 2, 3, 12, 16, 14, 39, 8, 22, 10, 4, 27, 15, 11, 7, 44, 33, 20, 40, 31, 34, 18, 21, 6, 49, 47, 38, 26].

Cmax: 3647.0

Elapsed time:  0.002991914749145508 seconds


In [75]:
# Gupta Heuristic
start_time = time.time()
sol = gupta_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Gupta.append("%.3f" % (deviation(Cmax, UB)))
Gupta.append(" %.3fs" % (elapsed_time))

Solution: [32, 21, 41, 26, 28, 19, 24, 31, 17, 48, 40, 13, 36, 42, 10, 5, 27, 3, 15, 37, 35, 14, 45, 18, 39, 8, 22, 49, 2, 4, 7, 6, 34, 20, 9, 47, 11, 0, 16, 44, 12, 33, 46, 1, 29, 30, 25, 23, 43, 38].

Cmax: 3627.0

Elapsed time:  0.0029909610748291016 seconds


In [76]:
# Palmer Heuristic
start_time = time.time()
sol = palmer_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Palmer.append("%.3f" % (deviation(Cmax, UB)))
Palmer.append(" %.3fs" % (elapsed_time))

Solution: [19, 17, 32, 41, 35, 37, 13, 42, 14, 43, 1, 48, 28, 3, 30, 24, 36, 21, 33, 15, 2, 45, 27, 5, 29, 11, 12, 31, 40, 39, 6, 10, 9, 7, 46, 0, 16, 8, 18, 25, 22, 20, 4, 44, 34, 23, 47, 26, 49, 38].

Cmax: 3415.0

Elapsed time:  0.0059833526611328125 seconds


In [77]:
# CDS Heuristic
start_time = time.time()
sol, Cmax = CDS_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

CDS.append("%.3f" % (deviation(Cmax, UB)))
CDS.append(" %.3fs" % (elapsed_time))

Solution: [17, 19, 36, 32, 24, 29, 48, 21, 43, 41, 30, 37, 28, 35, 11, 6, 13, 42, 1, 15, 33, 14, 3, 40, 27, 2, 22, 12, 39, 45, 7, 8, 4, 34, 16, 46, 31, 25, 10, 5, 9, 18, 20, 49, 47, 44, 0, 23, 26, 38].

Cmax: 3421.0

Elapsed time:  0.020940780639648438 seconds


In [78]:
# PKSE Heuristic
start_time = time.time()
sol, Cmax = PRSKE_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

PRSKE.append("%.3f" % (deviation(Cmax, UB)))
PRSKE.append(" %.3fs" % (elapsed_time))

Solution: [14, 49, 4, 8, 16, 12, 22, 1, 44, 15, 24, 42, 10, 37, 33, 35, 39, 21, 46, 45, 28, 27, 40, 2, 30, 20, 3, 19, 32, 18, 34, 11, 31, 13, 5, 17, 48, 23, 7, 29, 6, 9, 38, 25, 0, 47, 41, 36, 43, 26].

Cmax: 3900.0

Elapsed time:  0.005984783172607422 seconds


In [79]:
# Artificial_heuristic Heuristic
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

ARTIFICIAL.append("%.3f" % (deviation(Cmax, UB)))
ARTIFICIAL.append(" %.3fs" % (elapsed_time))

Solution: [41, 17, 43, 32, 19, 29, 36, 35, 48, 21, 24, 13, 33, 30, 28, 42, 37, 1, 3, 14, 2, 15, 45, 27, 8, 39, 22, 10, 16, 4, 12, 46, 9, 5, 11, 40, 31, 44, 20, 7, 25, 34, 6, 49, 18, 0, 47, 23, 38, 26].

Cmax: 3461.0

Elapsed time:  0.025282621383666992 seconds


### 100 jobs 10 machines

In [80]:
instance_0 = np.array(instances_100_10[0])
instance_0 = instance_0.T
UB = 5770

In [81]:
# NEH Algorithm
start_time = time.time()
sol, Cmax = neh_algorithm(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

NEH.append("%.3f" % (deviation(Cmax, UB)))
NEH.append(" %.3fs" % (elapsed_time))

Solution: [69, 5, 75, 79, 26, 19, 33, 6, 3, 97, 40, 66, 84, 72, 80, 85, 32, 95, 30, 24, 65, 83, 18, 0, 45, 8, 20, 31, 50, 94, 59, 38, 9, 34, 16, 90, 61, 86, 78, 41, 28, 87, 96, 7, 10, 63, 92, 27, 81, 76, 1, 4, 64, 57, 14, 73, 52, 17, 42, 70, 51, 68, 47, 23, 89, 46, 35, 43, 98, 77, 60, 55, 13, 91, 21, 67, 56, 15, 48, 37, 82, 99, 22, 88, 12, 25, 49, 44, 93, 54, 2, 71, 36, 39, 62, 29, 74, 53, 58, 11].

Cmax: 6062.0

Elapsed time:  6.391362428665161 seconds


In [84]:
# Greedy NEH Algorithm
start_time = time.time()
sol, Cmax = greedy_neh_algorithm(instance_0, 4,1)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

GNEH.append("%.3f" % (deviation(Cmax, UB)))
GNEH.append(" %.3fs" % (elapsed_time))

Solution: [76, 4, 57, 19, 43, 60, 34, 96, 24, 95, 69, 14, 75, 46, 65, 35, 59, 80, 94, 30, 66, 28, 16, 48, 1, 92, 20, 81, 90, 7, 50, 0, 82, 85, 40, 99, 10, 8, 78, 49, 31, 6, 84, 38, 72, 91, 21, 64, 9, 68, 79, 33, 93, 37, 61, 5, 54, 97, 88, 17, 47, 23, 45, 77, 18, 63, 15, 42, 32, 53, 22, 3, 70, 13, 26, 56, 67, 51, 27, 36, 44, 62, 83, 41, 39, 12, 87, 86, 29, 73, 74, 89, 52, 25, 55, 98, 58, 2, 71, 11].

Cmax: 5894.0

Elapsed time:  157.97121906280518 seconds


In [85]:
# Ham Heuristic
start_time = time.time()
sol, Cmax = ham_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Ham.append("%.3f" % (deviation(Cmax, UB)))
Ham.append(" %.3fs" % (elapsed_time))

Solution: [69, 43, 57, 60, 4, 20, 63, 92, 26, 51, 14, 76, 19, 38, 42, 30, 95, 86, 1, 90, 59, 34, 64, 16, 83, 49, 94, 82, 88, 61, 32, 33, 48, 80, 23, 7, 37, 40, 41, 5, 66, 78, 87, 79, 54, 56, 96, 91, 0, 21, 65, 22, 75, 3, 67, 24, 84, 31, 9, 13, 74, 47, 8, 10, 18, 72, 45, 97, 55, 46, 89, 17, 85, 70, 15, 2, 50, 53, 6, 98, 99, 93, 52, 27, 68, 28, 73, 77, 81, 62, 25, 12, 44, 29, 35, 58, 71, 36, 11, 39].

Cmax: 6255.0

Elapsed time:  0.003989696502685547 seconds


In [86]:
# Gupta Heuristic
start_time = time.time()
sol = gupta_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Gupta.append("%.3f" % (deviation(Cmax, UB)))
Gupta.append(" %.3fs" % (elapsed_time))

Solution: [36, 60, 34, 57, 92, 41, 14, 35, 48, 13, 50, 42, 69, 73, 30, 46, 59, 76, 27, 38, 68, 56, 23, 45, 55, 16, 86, 82, 88, 95, 33, 78, 72, 19, 61, 94, 66, 84, 7, 96, 5, 65, 79, 91, 54, 21, 0, 87, 31, 40, 10, 9, 67, 93, 80, 3, 53, 70, 22, 18, 52, 90, 8, 15, 75, 28, 77, 89, 24, 17, 64, 85, 32, 49, 47, 99, 62, 98, 6, 97, 83, 37, 1, 44, 81, 58, 74, 2, 4, 29, 39, 12, 25, 71, 43, 63, 20, 26, 11, 51].

Cmax: 6499.0

Elapsed time:  0.011967897415161133 seconds


In [87]:
# Palmer Heuristic
start_time = time.time()
sol = palmer_heuristic(instance_0)
Cmax = evaluate_sequence(sol, instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

Palmer.append("%.3f" % (deviation(Cmax, UB)))
Palmer.append(" %.3fs" % (elapsed_time))

Solution: [57, 69, 19, 43, 60, 63, 34, 14, 38, 30, 20, 59, 92, 5, 95, 56, 4, 94, 41, 1, 76, 79, 83, 82, 33, 48, 23, 66, 91, 96, 88, 16, 0, 80, 86, 54, 13, 64, 42, 78, 51, 61, 21, 7, 40, 84, 55, 32, 37, 46, 50, 72, 49, 97, 44, 75, 26, 10, 87, 65, 27, 9, 35, 17, 6, 18, 73, 89, 24, 3, 28, 31, 81, 90, 45, 85, 67, 15, 8, 62, 36, 93, 47, 98, 22, 52, 70, 53, 12, 77, 71, 99, 68, 2, 58, 74, 39, 11, 25, 29].

Cmax: 6028.0

Elapsed time:  0.006982088088989258 seconds


In [88]:
# CDS Heuristic
start_time = time.time()
sol, Cmax = CDS_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

CDS.append("%.3f" % (deviation(Cmax, UB)))
CDS.append(" %.3fs" % (elapsed_time))

Solution: [60, 34, 14, 20, 57, 59, 38, 56, 55, 69, 44, 16, 95, 43, 76, 90, 84, 63, 32, 19, 50, 5, 41, 88, 62, 46, 79, 80, 4, 40, 82, 30, 61, 78, 86, 23, 91, 94, 17, 48, 92, 96, 10, 1, 21, 81, 87, 33, 83, 6, 54, 45, 0, 13, 65, 15, 89, 97, 75, 85, 73, 31, 3, 18, 7, 35, 72, 37, 64, 9, 68, 24, 77, 51, 70, 67, 12, 93, 66, 8, 53, 26, 22, 52, 27, 28, 36, 49, 47, 42, 39, 99, 98, 74, 58, 2, 29, 71, 25, 11].

Cmax: 6209.0

Elapsed time:  0.045876264572143555 seconds


In [89]:
# PKSE Heuristic
start_time = time.time()
sol, Cmax = PRSKE_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

PRSKE.append("%.3f" % (deviation(Cmax, UB)))
PRSKE.append(" %.3fs" % (elapsed_time))

Solution: [22, 56, 74, 65, 67, 91, 21, 75, 83, 72, 15, 41, 45, 96, 13, 97, 34, 17, 24, 0, 23, 5, 84, 47, 66, 6, 89, 50, 2, 3, 64, 19, 7, 40, 32, 38, 31, 70, 81, 9, 1, 8, 29, 18, 51, 37, 87, 10, 12, 94, 49, 80, 48, 85, 46, 62, 30, 78, 92, 33, 79, 88, 59, 98, 55, 93, 61, 77, 4, 68, 99, 90, 52, 73, 54, 27, 26, 25, 82, 69, 35, 95, 63, 53, 42, 16, 20, 28, 60, 86, 58, 14, 43, 71, 39, 76, 44, 11, 57, 36].

Cmax: 7093.0

Elapsed time:  0.010969877243041992 seconds


In [90]:
# Artificial_heuristic Heuristic
start_time = time.time()
sol, Cmax = artificial_heuristic(instance_0)
elapsed_time = time.time() - start_time

print(f'Solution: {sol}.')
print(f'\nCmax: {Cmax}')
print("\nElapsed time: ", elapsed_time, "seconds")

ARTIFICIAL.append("%.3f" % (deviation(Cmax, UB)))
ARTIFICIAL.append(" %.3fs" % (elapsed_time))

Solution: [60, 14, 20, 34, 57, 69, 59, 44, 56, 19, 38, 55, 76, 43, 16, 84, 95, 30, 66, 63, 79, 86, 92, 4, 41, 61, 23, 82, 5, 96, 48, 46, 35, 88, 33, 50, 80, 72, 78, 94, 91, 83, 54, 21, 7, 1, 0, 97, 13, 65, 15, 6, 89, 75, 85, 17, 45, 18, 87, 10, 73, 31, 81, 40, 3, 12, 64, 37, 9, 32, 51, 8, 70, 62, 68, 93, 24, 67, 22, 26, 77, 52, 53, 42, 28, 98, 90, 27, 47, 49, 99, 36, 29, 74, 2, 58, 39, 71, 25, 11].

Cmax: 6207.0

Elapsed time:  0.06781697273254395 seconds


### Results

In [93]:
df_0.loc[0]=NEH
df_0.loc[1]=GNEH
df_0.loc[2]=Ham
df_0.loc[3]=Gupta
df_0.loc[4]=Palmer
df_0.loc[5]=CDS
df_0.loc[6]=PRSKE
df_0.loc[7]=ARTIFICIAL

In [94]:
# Displaying the results that we have got for the first instance of each benchmark
df_0

Heuristic 20-5 (Deviation) 20-5 (Time) 50-10 (Deviation)  \
0                   NEH            4.382      0.030s             6.744   
1            Greedy NEH            3.365      0.332s             7.405   
2                   Ham           10.876      0.000s            20.562   
3                 Gupta            9.233      0.000s            19.901   
4                Palmer            8.294      0.000s            12.893   
5                   CDS            8.764      0.004s            13.091   
6                 PRSKE           24.648      0.000s            28.926   
7  Artificial Heuristic            6.964      0.000s            14.413   

  50-10 (Time) 100-10 (Deviation) 100-10 (Time)  
0       0.730s              5.061        6.391s  
1      10.708s              2.149      157.971s  
2       0.003s              8.406        0.004s  
3       0.003s             12.634        0.012s  
4       0.006s              4.471        0.007s  
5       0.021s              7.608        0.046s  
6       0.006s             22.929        0.011s  
7       0.025s              7.574        0.068s